In [1]:
from werpy_weighted import werpy
# import werpy

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint
import logging
import os
import os.path as op

import torch
import torch.nn.functional as F
import numpy as np
import torchaudio
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
import whisper

In [5]:
from whisper.tokenizer import get_tokenizer
# https://github.com/openai/whisper/discussions/1041#discussioncomment-5224837
tokenizer = get_tokenizer(multilingual=True)  # use multilingual=True if using multilingual model
number_tokens = [
    i 
    for i in range(tokenizer.eot)
    if all(c in "0123456789" for c in tokenizer.decode([i]).removeprefix(" "))
]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

whisper_model = whisper.load_model('base.en').to(device)

In [7]:
split_txt_val_path = r'.\preprocessed_data\LJSpeech\val.txt'
val_uids = []
gt_transcripts_dict = {}
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])
        gt_transcripts_dict[line.split('|')[0]] = line.split('|')[-1]

In [8]:
syn_folder = r'.\output\0629lj\result\LJSpeech\wav\synthesized'

In [9]:
# uid = val_uids[0]
# print(torchaudio.info(op.join(syn_folder, f"{uid}.wav")))
# op.join(syn_folder, f"{uid}.wav")

In [10]:
%%time
uids = []
whisper_transcripts = []
gt_transcripts = []

for uid in tqdm(val_uids):
    uids.append(uid)
    output = whisper_model.transcribe(
            op.join(syn_folder, f"{uid}.wav"),
            suppress_tokens=[-1] + number_tokens,
    )
    whisper_transcript = output["text"]
    gt_transcript = gt_transcripts_dict[uid]

    whisper_transcripts.append(whisper_transcript)
    gt_transcripts.append(gt_transcript)


  0%|          | 0/512 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
lj_whisper_df = pd.DataFrame({
    'uid': uids,
    'whisper_transcripts': whisper_transcripts,
    'gt_transcripts': gt_transcripts,
})

In [22]:
lj_whisper_df.head()

,uid,whisper_transcripts,gt_transcripts
0,LJ047-0179,did nothing further in connection with the Os...,hosty did nothing further in connection with t...
1,LJ028-0176,Should we compare these ancient descriptions ...,should we compare these ancient descriptions o...
2,LJ032-0230,that the published pictures were the same as ...,that the published pictures were the same as t...
3,LJ016-0174,Having mentioned CalCraft's name.,"having mentioned calcraft's name,\n"
4,LJ016-0410,"nodded pleasantly to the governor, who stood ...","nodded pleasantly to the governor, who stood j..."


In [23]:
lj_whisper_df.to_csv(r".\jupyter_walkthrough\metrics\lj_whisper_baseen_df.csv", index=False)

In [3]:
import pandas as pd
lj_whisper_df = pd.read_csv(r".\jupyter_walkthrough\metrics\lj_whisper_df.csv")

In [4]:
lj_whisper_df.head()

,uid,whisper_transcripts,gt_transcripts
0,LJ047-0179,did nothing further in connection with the Os...,hosty did nothing further in connection with t...
1,LJ028-0176,Should we compare these ancient descriptions ...,should we compare these ancient descriptions o...
2,LJ032-0230,that the published pictures were the same as ...,that the published pictures were the same as t...
3,LJ016-0174,Having mentioned CalCraft's name.,"having mentioned calcraft's name,\n"
4,LJ016-0410,"nodded pleasantly to the governor, who stood ...","nodded pleasantly to the governor, who stood j..."


In [5]:
normalized_ref = werpy.normalize(
    lj_whisper_df['gt_transcripts'].values.tolist(),
)
normalized_hypo = werpy.normalize(
    lj_whisper_df['whisper_transcripts'].values.tolist(),
)

In [6]:
normalized_ref[0]

'hosty did nothing further in connection with the oswald case until after the assassination on november one nineteen sixtythree'

In [8]:
normalized_hypo[0]

'did nothing further in connection with the oswald case until after the assassination on november one nineteen sixtythree'

In [13]:
%%time
summary = werpy.summary(normalized_ref, normalized_hypo)
summary

CPU times: total: 109 ms
Wall time: 132 ms


,wer,ld,m,insertions,deletions,substitutions,inserted_words,deleted_words,substituted_words
0,0.157895,3,19,0,1,0,[],[hosty],[]
1,0.000000,0,15,0,0,0,[],[],[]
2,0.000000,0,28,0,0,0,[],[],[]
3,0.000000,0,4,0,0,0,[],[],[]
4,0.000000,0,22,0,0,0,[],[],[]
...,...,...,...,...,...,...,...,...,...
507,0.280000,7,25,1,0,1,[vice],[],"[(vicepresidential, presidential)]"
508,0.333333,3,9,0,1,0,[],[three],[]
509,0.000000,0,25,0,0,0,[],[],[]
510,0.000000,0,17,0,0,0,[],[],[]


In [16]:
I = summary['insertions'].sum()
D = summary['deletions'].sum()
S = summary['substitutions'].sum()
M = summary['m'].sum()

In [20]:
summary.to_csv(r".\jupyter_walkthrough\metrics\lj_wer_summary_baseen.csv", index=False)

In [17]:
wer = (I + D + S) / M

In [19]:
I, D, S

(104, 99, 279)

In [21]:
I/M, D/M, S/M

(0.012049588691924459, 0.011470281543274244, 0.03232533889468196)

In [18]:
wer

0.05584520912988066